# CNN Models

## Importing Required Libraries

In [3]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import shutil
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model

## Reseach Paper Model : Model 1

In [57]:
DefaultConv2D = partial(keras.layers.Conv2D,
 kernel_size=3, activation='relu', padding="SAME")

model_rp = keras.models.Sequential([
             DefaultConv2D(filters=16, input_shape=[128, 128, 3]),
             keras.layers.MaxPooling2D(pool_size=2),
             DefaultConv2D(filters=32),
             keras.layers.MaxPooling2D(pool_size=2),
             DefaultConv2D(filters=64),
             keras.layers.MaxPooling2D(pool_size=2),
             keras.layers.Dropout(0.5),
             keras.layers.Flatten(),
             keras.layers.Dense(units=7, activation='softmax'),
            ])

In [58]:
model_rp.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [59]:
model_rp.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_491 (Conv2D)         (None, 128, 128, 16)      448       
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 64, 64, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_492 (Conv2D)         (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_493 (Conv2D)         (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 16, 16, 64)      

In [60]:
batch_size = 64
train_dir = 'augmented'
test_dir = 'PData/test'
datagen = ImageDataGenerator(rescale = 1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical') # set as training data

validation_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 32738 images belonging to 7 classes.
Found 3006 images belonging to 7 classes.


In [61]:
r = model_rp.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 50
)

/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/239168063.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model_rp.fit_generator(


Epoch 1/50


2022-03-11 22:37:13.089457: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 1.3777 - accuracy: 0.4505

2022-03-11 22:37:51.636560: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - 52s 100ms/step - loss: 1.3777 - accuracy: 0.4505 - val_loss: 1.1741 - val_accuracy: 0.5686
Epoch 2/50
511/511 [==============================] - 51s 100ms/step - loss: 1.1218 - accuracy: 0.5665 - val_loss: 1.0756 - val_accuracy: 0.5842
Epoch 3/50
511/511 [==============================] - 49s 96ms/step - loss: 1.0022 - accuracy: 0.6172 - val_loss: 0.8918 - val_accuracy: 0.6763
Epoch 4/50
511/511 [==============================] - 49s 95ms/step - loss: 0.9168 - accuracy: 0.6534 - val_loss: 1.0551 - val_accuracy: 0.6077
Epoch 5/50
511/511 [==============================] - 49s 96ms/step - loss: 0.8545 - accuracy: 0.6748 - val_loss: 0.8047 - val_accuracy: 0.6953
Epoch 6/50
511/511 [==============================] - 50s 98ms/step - loss: 0.8015 - accuracy: 0.6980 - val_loss: 0.7539 - val_accuracy: 0.7313
Epoch 7/50
511/511 [==============================] - 49s 96ms/step - loss: 0.7703 - accuracy: 0.7083 - val_loss: 0.8347 - val_accuracy: 0.6909
E

In [63]:
#Confution Matrix and Classification Report
Y_pred = model_rp.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/171975285.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_rp.predict_generator(validation_generator, validation_generator.samples // batch_size+1)


Confusion Matrix
[[  44   18    3    7    7   20    0]
 [  25   94    5    6    1   21    1]
 [  25   13  151   11   33   97    0]
 [   2    2    0   22    2    7    0]
 [  25   10   39    6  154   98    2]
 [  21   34   76   30  105 1738    8]
 [   0    1    0    0    1    1   40]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.31      0.44      0.37        99
         bcc       0.55      0.61      0.58       153
         bkl       0.55      0.46      0.50       330
          df       0.27      0.63      0.38        35
         mel       0.51      0.46      0.48       334
          nv       0.88      0.86      0.87      2012
        vasc       0.78      0.93      0.85        43

    accuracy                           0.75      3006
   macro avg       0.55      0.63      0.57      3006
weighted avg       0.76      0.75      0.75      3006



## Modified Proposed Model: Model 2

In [6]:
# Copying folders of five classes which are majorly misclassfied in Model 1
# Will run this model only on those 5 classes
src_train = 'augmented'
dst_train = 'model_mini/augmented'
src_test = 'PData/test'
dst_test = 'model_mini/test'
for sk_class in ['akiec', 'bcc', 'bkl', 'df', 'mel']:
    shutil.copytree(f'{src_train}/{sk_class}', f'{dst_train}/{sk_class}', 
                dirs_exist_ok=False)
    shutil.copytree(f'{src_test}/{sk_class}', f'{dst_test}/{sk_class}',
                dirs_exist_ok=False)

In [68]:
model_rpm = keras.models.Sequential([
 DefaultConv2D(filters=32, input_shape=[128, 128, 3]),
 DefaultConv2D(filters=32),
 keras.layers.MaxPooling2D(pool_size=2),
 DefaultConv2D(filters=64),
 DefaultConv2D(filters=64),
 keras.layers.MaxPooling2D(pool_size=2),
 DefaultConv2D(filters=128),
 DefaultConv2D(filters=128),
 keras.layers.MaxPooling2D(pool_size=2),
 keras.layers.Flatten(),
 keras.layers.Dense(units=512, activation='relu'),
 keras.layers.Dense(units=512, activation='relu'),
 keras.layers.Dense(units=512, activation='relu'),
 keras.layers.Dropout(0.5),
 keras.layers.Dense(units=5, activation='softmax'),
])

In [69]:
model_rpm.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
model_rpm.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_506 (Conv2D)         (None, 128, 128, 32)      896       
                                                                 
 conv2d_507 (Conv2D)         (None, 128, 128, 32)      9248      
                                                                 
 max_pooling2d_50 (MaxPoolin  (None, 64, 64, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_508 (Conv2D)         (None, 64, 64, 64)        18496     
                                                                 
 conv2d_509 (Conv2D)         (None, 64, 64, 64)        36928     
                                                                 
 max_pooling2d_51 (MaxPoolin  (None, 32, 32, 64)       0         
 g2D)                                                

In [70]:

batch_size = 64
train_dir = 'model_mini/augmented'
test_dir = 'model_mini/test'
datagen = ImageDataGenerator(rescale = 1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical') # set as training data

validation_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 23397 images belonging to 5 classes.
Found 951 images belonging to 5 classes.


In [72]:
# The code is not in eror. The kernel was interrrupted after 11 Epochs since accuracy was decreasing
r = model_rpm.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 20
)

Epoch 1/20


/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/618141642.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model_rpm.fit_generator(


365/365 [==============================] - ETA: 0s - loss: 1.6095 - accuracy: 0.2016

2022-03-11 23:57:32.702134: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


365/365 [==============================] - 93s 254ms/step - loss: 1.6095 - accuracy: 0.2016 - val_loss: 1.6146 - val_accuracy: 0.0391
Epoch 2/20
365/365 [==============================] - 92s 253ms/step - loss: 1.6097 - accuracy: 0.1980 - val_loss: 1.6074 - val_accuracy: 0.1708
Epoch 3/20
365/365 [==============================] - 93s 256ms/step - loss: 1.6096 - accuracy: 0.1962 - val_loss: 1.6072 - val_accuracy: 0.1708
Epoch 4/20
365/365 [==============================] - 94s 257ms/step - loss: 1.6096 - accuracy: 0.1995 - val_loss: 1.6039 - val_accuracy: 0.1708
Epoch 5/20
365/365 [==============================] - 93s 254ms/step - loss: 1.6096 - accuracy: 0.1991 - val_loss: 1.6040 - val_accuracy: 0.3114
Epoch 6/20
365/365 [==============================] - 92s 252ms/step - loss: 1.6096 - accuracy: 0.1955 - val_loss: 1.6076 - val_accuracy: 0.1708
Epoch 7/20
365/365 [==============================] - 94s 258ms/step - loss: 1.6096 - accuracy: 0.1975 - val_loss: 1.6047 - val_accuracy: 0.1

KeyboardInterrupt: 

In [74]:
#Confution Matrix and Classification Report
Y_pred = model_rpm.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/240835413.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_rpm.predict_generator(validation_generator, validation_generator.samples // batch_size+1)


Confusion Matrix
[[  0  99   0   0   0]
 [  0 153   0   0   0]
 [  0 330   0   0   0]
 [  0  35   0   0   0]
 [  0 334   0   0   0]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.00      0.00      0.00        99
         bcc       0.16      1.00      0.28       153
         bkl       0.00      0.00      0.00       330
          df       0.00      0.00      0.00        35
         mel       0.00      0.00      0.00       334

    accuracy                           0.16       951
   macro avg       0.03      0.20      0.06       951
weighted avg       0.03      0.16      0.04       951



/Users/satyamvatts/miniforge3/envs/mlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/satyamvatts/miniforge3/envs/mlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/satyamvatts/miniforge3/envs/mlp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_sta

## ResNet50

In [75]:
IMAGE_SIZE = [128, 128]
en = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False, pooling='max')
x = Flatten()(en.output)
prediction = Dense(7, activation='softmax')(x)
model_keras = Model(inputs=en.input, outputs=prediction)
model_keras.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# view the structure of the model
model_keras.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 134, 134, 3)  0           ['input_7[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 64, 64, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 64, 64, 64)   256         ['conv1_conv[0][0]']       

 conv2_block3_2_conv (Conv2D)   (None, 32, 32, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 32, 32, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 32, 32, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_3_conv (Conv2D)   (None, 32, 32, 256)  16640       ['conv2_block3_2_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_2_conv (Conv2D)   (None, 16, 16, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 16, 16, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 16, 16, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_3_conv (Conv2D)   (None, 16, 16, 512)  66048       ['conv3_block3_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block2_2_conv (Conv2D)   (None, 8, 8, 256)    590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 8, 8, 256)   1024        ['conv4_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block2_2_relu[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block5_2_relu (Activatio  (None, 8, 8, 256)   0           ['conv4_block5_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_3_conv (Conv2D)   (None, 8, 8, 1024)   263168      ['conv4_block5_2_relu[0][0]']    
                                                                                                  
 conv4_block5_3_bn (BatchNormal  (None, 8, 8, 1024)  4096        ['conv4_block5_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_add (Add)         (None, 8, 8, 1024)   0           ['conv4_block4_out[0][0]',       
                                                                  'conv4_block5_3_bn[0][0]']      
          

 conv5_block2_2_relu (Activatio  (None, 4, 4, 512)   0           ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 4, 4, 2048)   1050624     ['conv5_block2_2_relu[0][0]']    
                                                                                                  
 conv5_block2_3_bn (BatchNormal  (None, 4, 4, 2048)  8192        ['conv5_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_add (Add)         (None, 4, 4, 2048)   0           ['conv5_block1_out[0][0]',       
                                                                  'conv5_block2_3_bn[0][0]']      
          

In [76]:
batch_size = 64
train_dir = 'augmented'
test_dir = 'PData/test'
datagen = ImageDataGenerator(rescale = 1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical') # set as training data

validation_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

Found 32738 images belonging to 7 classes.
Found 3006 images belonging to 7 classes.


In [77]:
r = model_keras.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 20
)

Epoch 1/20


/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/2682878022.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model_keras.fit_generator(
2022-03-12 00:21:28.325087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 1.2636 - accuracy: 0.6627

2022-03-12 00:27:55.569215: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - 409s 776ms/step - loss: 1.2636 - accuracy: 0.6627 - val_loss: 2.2573 - val_accuracy: 0.0119
Epoch 2/20
511/511 [==============================] - 392s 766ms/step - loss: 0.8318 - accuracy: 0.7288 - val_loss: 5.8635 - val_accuracy: 0.0489
Epoch 3/20
511/511 [==============================] - 411s 804ms/step - loss: 0.7528 - accuracy: 0.7762 - val_loss: 4281.9741 - val_accuracy: 0.0146
Epoch 4/20
511/511 [==============================] - 426s 833ms/step - loss: 0.6243 - accuracy: 0.7855 - val_loss: 2.0820 - val_accuracy: 0.7041
Epoch 5/20
511/511 [==============================] - 446s 873ms/step - loss: 0.4214 - accuracy: 0.8445 - val_loss: 1.9572 - val_accuracy: 0.7089
Epoch 6/20
511/511 [==============================] - 458s 897ms/step - loss: 0.3880 - accuracy: 0.8773 - val_loss: 4.7839 - val_accuracy: 0.2052
Epoch 7/20
511/511 [==============================] - 466s 912ms/step - loss: 0.2543 - accuracy: 0.9075 - val_loss: 2.2940 - val_acc

In [78]:
#Confution Matrix and Classification Report
Y_pred = model_keras.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/2922779573.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_keras.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
2022-03-12 02:55:04.472772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Confusion Matrix
[[  76    4   10    0    3    5    1]
 [  57   51   15    2    3   24    1]
 [  53    4  181    0   23   69    0]
 [   8    1    1   15    3    7    0]
 [  35   16   83    0  125   75    0]
 [ 173   66  327    7  202 1220   17]
 [   0    2    1    0    5    1   34]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.19      0.77      0.30        99
         bcc       0.35      0.33      0.34       153
         bkl       0.29      0.55      0.38       330
          df       0.62      0.43      0.51        35
         mel       0.34      0.37      0.36       334
          nv       0.87      0.61      0.71      2012
        vasc       0.64      0.79      0.71        43

    accuracy                           0.57      3006
   macro avg       0.47      0.55      0.47      3006
weighted avg       0.69      0.57      0.60      3006



## Inception V3

In [79]:
# Re-size images utilize ResNet Architecture
IMAGE_SIZE = [128, 128]

# EffecientNet Model
en = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False, pooling='max')

# don't train existing weights
# for layer in vgg.layers:
#   layer.trainable = False
# our layers - you can add more if you want

x = Flatten()(en.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(7, activation='softmax')(x)

# create a model object
model_keras2 = Model(inputs=en.input, outputs=prediction)

model_keras2.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)
# view the structure of the model
model_keras2.summary()

batch_size = 64
train_dir = 'augmented'
test_dir = 'PData/test'
datagen = ImageDataGenerator(rescale = 1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical') # set as training data

validation_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

r = model_keras2.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples//batch_size,
    validation_steps=validation_generator.samples//batch_size,
    validation_data = validation_generator,
    epochs = 20
)

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_512 (Conv2D)            (None, 63, 63, 32)   864         ['input_8[0][0]']                
                                                                                                  
 batch_normalization_470 (Batch  (None, 63, 63, 32)  96          ['conv2d_512[0][0]']             
 Normalization)                                                                                   
                                                                                            

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_480 (Batch  (None, 13, 13, 96)  288         ['conv2d_522[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_481 (Batch  (None, 13, 13, 32)  96          ['conv2d_523[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_475 (Activation)    (None, 13, 13, 64)   0           ['batch_normalization_475[0][0]']
                                                                                                  
 activatio

 activation_492 (Activation)    (None, 13, 13, 64)   0           ['batch_normalization_492[0][0]']
                                                                                                  
 conv2d_532 (Conv2D)            (None, 13, 13, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_535 (Conv2D)            (None, 13, 13, 96)   55296       ['activation_492[0][0]']         
                                                                                                  
 batch_normalization_490 (Batch  (None, 13, 13, 48)  144         ['conv2d_532[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_493 (Batch  (None, 13, 13, 96)  288         ['conv2d_535[0][0]']             
 Normaliza

 mixed3 (Concatenate)           (None, 6, 6, 768)    0           ['activation_496[0][0]',         
                                                                  'activation_499[0][0]',         
                                                                  'max_pooling2d_55[0][0]']       
                                                                                                  
 conv2d_546 (Conv2D)            (None, 6, 6, 128)    98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_504 (Batch  (None, 6, 6, 128)   384         ['conv2d_546[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_504 (Activation)    (None, 6, 6, 128)    0           ['batch_normalization_504[0][0]']
          

                                                                                                  
 batch_normalization_514 (Batch  (None, 6, 6, 160)   480         ['conv2d_556[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_514 (Activation)    (None, 6, 6, 160)    0           ['batch_normalization_514[0][0]']
                                                                                                  
 conv2d_557 (Conv2D)            (None, 6, 6, 160)    179200      ['activation_514[0][0]']         
                                                                                                  
 batch_normalization_515 (Batch  (None, 6, 6, 160)   480         ['conv2d_557[0][0]']             
 Normalization)                                                                                   
          

...

                                                                                                  
 activation_562 (Activation)    (None, 2, 2, 384)    0           ['batch_normalization_562[0][0]']
                                                                                                  
 batch_normalization_563 (Batch  (None, 2, 2, 192)   576         ['conv2d_605[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_555 (Activation)    (None, 2, 2, 320)    0           ['batch_normalization_555[0][0]']
                                                                                                  
 mixed9_1 (Concatenate)         (None, 2, 2, 768)    0           ['activation_557[0][0]',         
                                                                  'activation_558[0][0]']         
          

/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/4131774341.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model_keras2.fit_generator(
2022-03-12 02:55:39.168887: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 0.6278 - accuracy: 0.7762

2022-03-12 02:59:59.801665: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - 279s 530ms/step - loss: 0.6278 - accuracy: 0.7762 - val_loss: 1.9831 - val_accuracy: 0.4674
Epoch 2/20
511/511 [==============================] - 288s 564ms/step - loss: 0.2798 - accuracy: 0.9006 - val_loss: 2.1588 - val_accuracy: 0.6726
Epoch 3/20
511/511 [==============================] - 305s 596ms/step - loss: 0.2337 - accuracy: 0.9180 - val_loss: 4.4688 - val_accuracy: 0.6953
Epoch 4/20
511/511 [==============================] - 311s 608ms/step - loss: 0.1487 - accuracy: 0.9470 - val_loss: 3.3074 - val_accuracy: 0.7086
Epoch 5/20
511/511 [==============================] - 325s 635ms/step - loss: 0.1118 - accuracy: 0.9608 - val_loss: 4.3106 - val_accuracy: 0.6980
Epoch 6/20
511/511 [==============================] - 327s 640ms/step - loss: 0.0878 - accuracy: 0.9697 - val_loss: 1.7828 - val_accuracy: 0.7058
Epoch 7/20
511/511 [==============================] - 325s 635ms/step - loss: 0.0834 - accuracy: 0.9717 - val_loss: 1.7573 - val_accura

In [80]:
#Confution Matrix and Classification Report
Y_pred = model_keras2.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, y_pred))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(validation_generator.classes, y_pred, target_names=target_names))

/var/folders/g6/xjmqzxt515z147xzc4yys5j40000gn/T/ipykernel_13948/3472471282.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_keras2.predict_generator(validation_generator, validation_generator.samples // batch_size+1)
2022-03-12 04:40:12.538615: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Confusion Matrix
[[  20    1    1    0    0   77    0]
 [   2   12    1    0    0  138    0]
 [   5    0   26    2    8  288    1]
 [   1    0    0    9    0   25    0]
 [   2    0   16    0   48  268    0]
 [   0    1    8    1    9 1990    3]
 [   0    0    0    1    3    4   35]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.67      0.20      0.31        99
         bcc       0.86      0.08      0.14       153
         bkl       0.50      0.08      0.14       330
          df       0.69      0.26      0.37        35
         mel       0.71      0.14      0.24       334
          nv       0.71      0.99      0.83      2012
        vasc       0.90      0.81      0.85        43

    accuracy                           0.71      3006
   macro avg       0.72      0.37      0.41      3006
weighted avg       0.70      0.71      0.63      3006

